## Opening Images

As we have done in previous examples, we use the micasense.capture class to open, radiometrically correct, and visualize the 10 bands of a RedEdge-MX Dual (Red+Blue) capture.

First, we'll load the `autoreload` extension.  This lets us change underlying code (such as library functions) without having to reload the entire workbook and kernel. This is useful in this workbook because the cell that runs the alignment can take a long time to run, so with `autoreload` extension we can update the code after the alignment step for analysis and visualization without needing to re-compute the alignments each time.

In [1]:
%load_ext autoreload
%autoreload 2

This notebook aims to create a calibration curve for radiometric correction under different illumination conditions. Without effective radiometric calibration, drone imagery heavily suffers from sun illumination affected by time of day, cloud, and other weather conditions. UAV sensors are vulnerable to vibration and wind effect during flight. Their radiometric properties may also be impacted by other environmental variations such as temperature, solar illumination, and weather conditions. Therefore, even with calibrated sensors, in situ calibration is often needed to improve accuracies of quantitative applications.

The most common in situ calibration approach is the Empirical Line Method (ELM) to build the relationships between the reflectance of calibrated targets and their top-of-atmosphere radiance (at sensor radiance). It requires the real-time reflectance measurements of multiple artificial or natural targets. Some studies explored the irradiance-based ELM calibration by considering the aerosol effects in the relationship. This empirical calibration is often time consuming and not practical efficient. The real-time field spectra may not be needed when a reference standard is employed in field. The RedEdge system utilises a pre-calibration reflectance panel (CRP) set up in field and a Downwelling Light Sensor (DLS) mounted atop a drone to capture the hemispherical irradiance during the flight. The current procedure of RedEdge image correction mostly utilises the CRP panel itself and some studies have reported the relatively higher percent errors of its radiometric calibration. With the bar code embedded on the panel, the CRP reflectance can be automatically recognised in image processing packages.

## Radiometric calibration model

A radiometric calibration model has been has been developed to convert the raw pixel values into absolute spectral radiance:

$$ L = V(x,y) \cdot \frac{a_1}{g} \cdot \frac{p - p_{BL}}{t_e + a_2 y -a_3 t_e y}$$

Where,

- $p$ is the normalised raw pixel value (0 to 1 by dividing DN with the number of bits)
- $p_{BL}$ is the normalised black level value (can be found in metadata tags). This tag encodes an array of 4 values, which should be averaged to compute a black level offset which can be applied to all pixels
- $a_1, a_2, a_3$ are the radiometric calibration coefficients (to convert DN to radiance)
- $V(x,y)$ is the vignette polynomial function for pixel location (x,y). See "Vignette Model" section
- $t_e$ is the image exposure time
- $g$ is the sensor gain setting (can be found in metadata tags)
- $x, y$ are the pixel column and row number, respectively
- $L$ is the spectral radiance in $W/m^2/sr/nm$

## Vignette Model

The RedEdge uses a radial vignette model to correct for the fall-off in light sensitivity that occurs in pixels further from the center of the image. To apply the model, first read $cx, cy$ and the six polynomial coefficients from the image metadata, then compute the formula below to find a correction scale factor for each pixel intensity

$r = \sqrt{(x-c_x)^2 + (y-c_y)^2}$

$k = 1 + k_0 * r + k_1 * r^2 + k_2 * r^3 + k_3 * r^4 + k^4 * r^5 + k_5 * r^6$

$I_{corrected}(x,y) = \frac{I(x,y)}{k}$

Where,

- $r$ is the distance of the pixel (x,y) from the vignette center, in pixels
- $(x,y)$ is the coordinate of the pixel being corrected
- $k$ is the correction factor by which the raw pixel value should be divided to correct for vignette
- $I(x,y)$ is the original intensity of pixel at $x,y$
- $I_{corrected}(x,y)$ is the corrected intensity of pixel at x,y

In the radiance model above, $V(x,y)$ is equal to $1/k$

## Radiometric Correction

The radiance after calibration is further converted to surface reflectance for spectral analysis. Assisted with a pre-calibrated CRP panel, the surface reflectance ($\rho$) of the RedEdge-M image can be simply converted as:

$$ \rho(\lambda) = \frac{\rho_{CRP}}{L_{CRP}} \times L(\lambda) \quad (1)$$

Where,
- $\frac{\rho_{CRP}}{L_{CRP}}$ is the reflectance calibration factor to convert radiance of an image to reflectance
- $\rho_{CRP}$ is the pre-calibrated reflectance of the CRP panel
- $L_{CRP}$ is the average radiance of the CRP panel
- $\rho(\lambda)$ is the reflectance of an image of interest
- $L$ is the radiance of an image of interest

These micasense imageprocessing tool scan the QR code and identify a squared area within the CRP panel, then calculate the average radiance. The reflectance calibration factor us aookued to all pixels of the image to extract surface reflectance

This approach is easy to use and works well when the relative biophysical quantities are of major concern. However, the radiometric correction relies on one single panel with a pre-determined constant coefficient for each band. In field, the panel's reflectance may be affected by solar illumination and path scattering that vary with time and atmospheric and environmental conditions. Therefore, a more rigid radiometric correction method is needed to better calculate the reflective values of the images

## At-sensor DLS Radiometric Correction

The camera takes the CRP calibration images on ground before and after a flight mission. The image records the radiance reflected out of the CRP panel, which can be transformed to the incoming irradiances of each band given the known reflectance of the diffuse material. At the same time, the Downwelling Light Sensor (DLS) records the incoming irradiance in each band at a unit $W/m^2/nm$. The irradiance recordings are stored as metadata of each image.

Influenced by the mechanical differences between the two sensors and environmental scattering, the two irradiance recordings would not be identical but should follow a linear relationship:

$$ Irr_{CRP}(\lambda) = a \times Irr_{DLS}(\lambda) + b $$

Where,

- $Irr_{CRP}(\lambda)$ represent the camera-recorded irradiance reaching the CRP
- $Irr_{DLS}(\lambda)$ represent the DLS-recorded irradiance atop the drone (Note the DLS is installed upward facing the direct sky)
- coefficients $a,b$ represents the differences between the two sensors

The CRP is made of diffuser materials that are supposed to have equal radiance in all directions. The $Irr_{CRP}(\lambda)$ can thus be simplified as the CRP radiance multiplied by $\pi$. Similarly, the DLS radiance can be calculated as the $Irr_{DLS}(\lambda)$ divided by $\pi$.

For a known reflectance of the diffuse material ($\rho_{CRP}$ which varies from 0 to 1), the corresponding radiance is thus $L_{CRP}$, thus the irradiance (i.e. 100% reflectance) is thus: $\frac{L_{CRP}(\lambda)}{\rho_{CRP}}$

$$ \frac{\pi L_{CRP}(\lambda)}{\rho_{CRP}} = a \times \pi L_{DLS}(\lambda) + b $$

Dividing by $\pi$

$$ \frac{L_{CRP}(\lambda)}{\rho_{CRP}} = a \times L_{DLS}(\lambda) + \frac{b}{\pi} \quad (2)$$

Substituting (2) in (1),

and $\rho (\lambda) = \frac{\rho_{CRP}}{L_{CRP}} \times L(\lambda)$, thus $L(\lambda) = \frac{\rho(\lambda) L_{CRP}}{\rho_{CRP}}$

$$\rho_{cor}(\lambda) = \frac{a}{1- \frac{b \times \rho_{CRP}}{\pi L_{CRP}(\lambda)}} \times \rho (\lambda)$$

where $Cor(\lambda) = \frac{a}{1- \frac{b \times \rho_{CRP}}{\pi L_{CRP}(\lambda)}}$ is the correction factor applied to each flight mission. Since $\rho_{CRP}$ is pre-determined, and $L_{CRP}$ is mission specific and is extracted from the panel calibration image of a flight mission. We just need to solve for $a,b$ from the CRP_DLS irradiance regression. Both CRP and DLS are made of diffuse materials and therefore possess a common relationshop between their irradiance recordings. The study relies on all calibration data collected in multiple flights at different study sites to build the CRP-DLS relationship.

In [1]:
import micasense.imageset as imageset
import micasense.capture as capture
import os, glob
import multiprocessing

# Import panels

In [30]:

def get_panels(dir,search_n=5):
    """
    :param dir(str): directory where images are stored
    search for the first 5 and last 5 captures to detect panel
    returns a list of captures in band order
    """
    number_of_files = len(glob.glob(os.path.join(dir,'IMG_*.tif')))//10 #divide by 10 since each capture has 10 bands
    last_file_number = number_of_files - 1 #since index of image starts from 0

    first_few_panels_fp = [glob.glob(os.path.join(dir,'IMG_000{}_*.tif'.format(str(i)))) for i in range(search_n)]
    last_few_panels_fp = [glob.glob(os.path.join(dir,'IMG_{}_*.tif'.format(str(last_file_number-i).zfill(4)))) for i in reversed(range(search_n))]
    panels_fp = first_few_panels_fp + last_few_panels_fp
    panels_list = []

    for f in panels_fp:
        cap_dict = {i+1:None for i in range(10)} # in order to order the files by band order, otherwise IMG_1 and IMG_10 are consecutive
        for cap in f:
            cap_dict[int(cap.split('_')[-1].replace('.tif',''))] = cap
        panels_list.append(list(cap_dict.values()))
    return panels_list
    # panelCaps = [capture.Capture.from_filelist(f) for f in panels_fp] # list of captures
    # []

def import_panel_reflectance(panelNames):
    """
    :param PanelNames (list of str): full file path of panelNames
    For panel images, efforts will be made to automatically extract the panel information, 
    panel images are not warped and the QR codes are individually detected for each band
    """
    if panelNames is not None:
        panelCap = capture.Capture.from_filelist(panelNames)
    else:
        panelCap = None
    if panelCap is not None:
        if panelCap.panel_albedo() is not None:
            panel_reflectance_by_band = panelCap.panel_albedo()
        else:
            print("Panel reflectance not detected by serial number")
            panel_reflectance_by_band = None 
    else:
        panel_reflectance_by_band = None 
    
    return panel_reflectance_by_band

def import_panel_irradiance(panelNames,panel_reflectance_by_band):
    """
    :param PanelNames (list of str): full file path of panelNames
    :param panel_reflectance_by_band (list of float): reflectance values ranging from 0 to 1. 
    Import this value so we don't have to keep detecting the QR codes repeatedly if the QR panels are the same
    For panel images, efforts will be made to automatically extract the panel information, 
    panel images are not warped and the QR codes are individually detected for each band
    """
    if panelNames is not None:
        panelCap = capture.Capture.from_filelist(panelNames)
        panel_irradiance = panelCap.panel_irradiance(panel_reflectance_by_band)
        return panel_irradiance
    else:
        print("Panels not found")
        return None


In [31]:
panels_fp = get_panels(dir = r"F:\1stSur6Apr\F1\RawImg")
panels_fp

[['F:\\1stSur6Apr\\F1\\RawImg\\IMG_0000_1.tif',
  'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0000_2.tif',
  'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0000_3.tif',
  'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0000_4.tif',
  'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0000_5.tif',
  'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0000_6.tif',
  'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0000_7.tif',
  'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0000_8.tif',
  'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0000_9.tif',
  'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0000_10.tif'],
 ['F:\\1stSur6Apr\\F1\\RawImg\\IMG_0001_1.tif',
  'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0001_2.tif',
  'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0001_3.tif',
  'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0001_4.tif',
  'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0001_5.tif',
  'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0001_6.tif',
  'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0001_7.tif',
  'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0001_8.tif',
  'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0001_9.tif',
  'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0001_10.tif'],
 ['F:\\1stSur6Apr\\F1\\RawImg\\IMG_0

In [26]:
list(panels_fp[-1].values())

['F:\\1stSur6Apr\\F1\\RawImg\\IMG_0403_1.tif',
 'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0403_2.tif',
 'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0403_3.tif',
 'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0403_4.tif',
 'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0403_5.tif',
 'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0403_6.tif',
 'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0403_7.tif',
 'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0403_8.tif',
 'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0403_9.tif',
 'F:\\1stSur6Apr\\F1\\RawImg\\IMG_0403_10.tif']